# Singular Vector Decomposition 🔹

Singular Vector Decomposition is a very popular collaborative filtering technique for RecSys problems. It has been used a lot in the past when there was a boom in RecSys (Netflix Prize). It's still a pretty damn good model. We'll be exploring how to use this on our competition here.

#### If you liked this notebook, please leave an upvote!

# Problems 🔹

1. One of the problem of SVD for this task has been that it has implicit data (only positive outcomes) rather than explicit feedback (variation of at least two outcomes - yes/no, 1/0, 1-5 etc). The major task at hand is to convert the implicit feedback to explicit feedback.

2. The other bottleneck would have been the size of the user-item matrix which would have been too huge to fit in memory if we use the scipy svd. I'll use my recsys library where I had implemented the cython backed iterative version of svd or [FunkSVD](https://sifter.org/simon/journal/20061211.html) as it is famously called.

**Negative Sampling:** I experimented with different negative sampling policies but could not achieve any clear improvement over the existing methods. Hence I'll not include negative sampling mechanism here. Feel free to experiment.

#### reco (https://github.com/mayukh18/reco) 
using RecSys library **reco**. 

In [1]:
# It is available on pypi but I haven't updated the release there. So install from github
!pip install git+https://github.com/mayukh18/reco

  Cloning https://github.com/mayukh18/reco to /tmp/pip-req-build-xj9pxf81
  Running command git clone --filter=blob:none -q https://github.com/mayukh18/reco /tmp/pip-req-build-xj9pxf81
  Resolved https://github.com/mayukh18/reco to commit 7f0bfc40edc6c21afef4314ee30605766281990b
  Preparing metadata (setup.py) ... - \ done
  Created wheel for reco: filename=reco-0.2.1-cp37-cp37m-linux_x86_64.whl size=9868227 sha256=df02d38abe2146990847a497b40973da1453a47390a97adf46084fbe0afc0239
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0skr1il/wheels/6b/77/06/3fabb6de467036024e54b62021cc5d1dc8cf3a300b17c43089
Successfully built reco


In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os
import glob
import reco
from tqdm import tqdm
import datetime
from collections import Counter

# Forming Train Set 🔹

We'll keep 2 weeks as train and the last week as validation.

In [3]:
data = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", dtype={'article_id':str})
data["t_dat"] = pd.to_datetime(data["t_dat"])
data.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [4]:
print("All Transactions Date Range: {} to {}".format(data['t_dat'].min(), data['t_dat'].max()))

data["t_dat"] = pd.to_datetime(data["t_dat"])
train1 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,8)) & (data['t_dat'] < datetime.datetime(2020,9,16))]
train2 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,1)) & (data['t_dat'] < datetime.datetime(2020,9,8))]
train3 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,23)) & (data['t_dat'] < datetime.datetime(2020,9,1))]
train4 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,15)) & (data['t_dat'] < datetime.datetime(2020,8,23))]

val = data.loc[data["t_dat"] >= datetime.datetime(2020,9,16)]

All Transactions Date Range: 2018-09-20 00:00:00 to 2020-09-22 00:00:00


In [5]:
# List of all purchases per user (has repetitions)
positive_items_per_user1 = train1.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user2 = train2.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user3 = train3.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user4 = train4.groupby(['customer_id'])['article_id'].apply(list)

# Implicit to Explicit Feedback 🔹

There can be multiple ways to convert the implicit data to explicit. But all ways have the same fundamental idea. We'll treat the purchase of an unpopular item to be of high explicit feedback and that of a popular item to be of low explicit feedback. Also, if an user buys a lot of items, that can weigh down the importance of one single item.

Some ideas:

1. user-item purchase count weighed down by time decaying popularity of the item as introduced in https://www.kaggle.com/mayukh18/time-decaying-popularity-benchmark-0-0216 (We will use this in this notebook)
2. user-item purchase count weighed down by the product of total purchase count of the item and total purchase count of the user.

Note: No idea is the absolute perfect. As you will combine different heuristic approaches with the SVD you'll realize different conversion techniques are favourable for different heuristics approaches.

In [6]:
train = pd.concat([train1, train2, train3, train4], axis=0)

#time decay popularity of each article
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,16) - x).days**2)
popular_items_group = train.groupby(['article_id'])['pop_factor'].sum()

# purchase count of each article
items_total_count = train.groupby(['article_id'])['article_id'].count()
# purchase count of each user
users_total_count = train.groupby(['customer_id'])['customer_id'].count()


train['feedback'] = 1
train = train.groupby(['customer_id', 'article_id']).sum().reset_index()
train['feedback'] = train.apply(lambda row: row['feedback']/popular_items_group[row['article_id']], axis=1)

train['feedback'] = train['feedback'].apply(lambda x: 5.0 if x>5.0 else x)
train.drop(['price', 'sales_channel_id'], axis=1, inplace=True)

# shuffling
train = train.sample(frac=1).reset_index(drop=True)
train['feedback'].describe()

count    1.044811e+06
mean     8.252942e-01
std      1.455205e+00
min      5.373625e-03
25%      6.460413e-02
50%      1.771086e-01
75%      6.486035e-01
max      5.000000e+00
Name: feedback, dtype: float64

Basic time decaying popularity to combine it with the SVD.

In [7]:
train_pop = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,1)) & (data['t_dat'] < datetime.datetime(2020,9,16))]
train_pop['pop_factor'] = train_pop['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,16) - x).days)
popular_items_group = train_pop.groupby(['article_id'])['pop_factor'].sum()

_, popular_items = zip(*sorted(zip(popular_items_group, popular_items_group.keys()))[::-1])

train_pop['pop_factor'].describe()

count    557958.000000
mean          0.200478
std           0.207752
min           0.066667
25%           0.083333
50%           0.125000
75%           0.200000
max           1.000000
Name: pop_factor, dtype: float64

Below part is a cool new addition which gives a small bump. Did not add it to the validation part but it has the same effect there too. It predicts the most frequent next item bought by an user after a particular item.

In [8]:
def get_most_freq_next_item(user_group):
    next_items = {}
    for user in tqdm(user_group.keys()):
        items = user_group[user]
        for i,item in enumerate(items[:-1]):
            if item not in next_items:
                next_items[item] = []
            if item != items[i+1]:
                next_items[item].append(items[i+1])

    pred_next = {}
    for item in next_items:
        if len(next_items[item]) >= 5:
            most_common = Counter(next_items[item]).most_common()
            ratio = most_common[0][1]/len(next_items[item])
            if ratio >= 0.1:
                pred_next[item] = most_common[0][0]
            
    return pred_next

user_group = train.groupby(['customer_id'])['article_id'].apply(list)
pred_next = get_most_freq_next_item(user_group)

100%|██████████| 254618/254618 [00:02<00:00, 102682.51it/s]


# SVD Training 🔹

In [9]:
from reco.recommender import FunkSVD
from reco.metrics import rmse

# k = number of dimensions of the latent embedding. formatizer dict takes in names of the columns
# for user, item and values/feedback/ratings respectively.

svd = FunkSVD(k=8, learning_rate=0.008, regularizer = .01, iterations = 80, method = 'stochastic', bias=True)
svd.fit(X=train, formatizer={'user':'customer_id', 'item':'article_id', 'value':'feedback'},verbose=True)

Epoch 0: Error: 0.6739707229979606
Epoch 1: Error: 0.47521176958192185
Epoch 2: Error: 0.40657986172888816
Epoch 3: Error: 0.36592138199234925
Epoch 4: Error: 0.33794449119059233
Epoch 5: Error: 0.31682703156247055
Epoch 6: Error: 0.29984902840474736
Epoch 7: Error: 0.2855742521715772
Epoch 8: Error: 0.27321171105644226
Epoch 9: Error: 0.2622580359148723
Epoch 10: Error: 0.2523628217724591
Epoch 11: Error: 0.24330339390304692
Epoch 12: Error: 0.23492637503712135
Epoch 13: Error: 0.22712942728784902
Epoch 14: Error: 0.2198278428683797
Epoch 15: Error: 0.21296703746023632
Epoch 16: Error: 0.20650436674366615
Epoch 17: Error: 0.2004077968284915
Epoch 18: Error: 0.19465644787357794
Epoch 19: Error: 0.1892257566316674
Epoch 20: Error: 0.18409229205394678
Epoch 21: Error: 0.17923679825229905
Epoch 22: Error: 0.17464186958360928
Epoch 23: Error: 0.17029294736049497
Epoch 24: Error: 0.16617312796639205
Epoch 25: Error: 0.16226931263636635
Epoch 26: Error: 0.15856579157998063
Epoch 27: Error: 0

# Validation 🔹

This is mostly based on [this notebook](https://www.kaggle.com/mayukh18/time-decaying-popularity-benchmark-0-0216) where I have used same pipeline. Will use SVD for re-ranking. Will read all the data anew and train a new model on our new train set with new date ranges for submission. This will align us with the aforementioned notebook.

In [10]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [11]:
positive_items_val = val.groupby(['customer_id'])['article_id'].apply(list)
val_users = positive_items_val.keys()
val_items = []

for i,user in tqdm(enumerate(val_users)):
    val_items.append(positive_items_val[user])
    
print("Total users in validation:", len(val_users))

68984it [00:00, 145664.60it/s]

Total users in validation: 68984


#### Normal way of prediction without the SVD reranking

In [12]:
from collections import Counter
outputs = []
cnt = 0

popular_items = list(popular_items)

for user in tqdm(val_users):
    user_output = []
    if user in positive_items_per_user1.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user1[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    if user in positive_items_per_user2.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user2[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    if user in positive_items_per_user3.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user3[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    if user in positive_items_per_user4.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user4[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    
    user_output += [pred_next[item] for item in user_output if item in pred_next and pred_next[item] not in user_output]      
    
    user_output += list(popular_items[:12 - len(user_output)])
    outputs.append(user_output)
    
print("mAP Score on Validation set:", mapk(val_items, outputs))

100%|██████████| 68984/68984 [00:05<00:00, 11977.13it/s]


mAP Score on Validation set: 0.024402599558659414


#### Now, prediction WITH the SVD reranking!

In [13]:
from collections import Counter
outputs = []
cnt = 0

popular_items = list(popular_items)
userindexes = {svd.users[i]:i for i in range(len(svd.users))}

for user in tqdm(val_users):
    user_output = []
    if user in positive_items_per_user1.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user1[user]).most_common()}
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = svd.items.index(k)
                pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    if user in positive_items_per_user2.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user2[user]).most_common()}
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = svd.items.index(k)
                pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    if user in positive_items_per_user3.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user3[user]).most_common()}
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = svd.items.index(k)
                pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    if user in positive_items_per_user4.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user4[user]).most_common()}
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = svd.items.index(k)
                pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    user_output += [pred_next[item] for item in user_output if item in pred_next and pred_next[item] not in user_output]      
    
    user_output += list(popular_items[:12 - len(user_output)])
    outputs.append(user_output)
    
print("mAP Score on Validation set:", mapk(val_items, outputs))

100%|██████████| 68984/68984 [02:02<00:00, 564.92it/s]


mAP Score on Validation set: 0.02539883499870202


Decent jump of 0.001 I would say with 4 weeks of training data!

# Submission

We will do all the same things all over again just with different time periods.

In [14]:
train1 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,16)) & (data['t_dat'] < datetime.datetime(2020,9,23))]
train2 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,8)) & (data['t_dat'] < datetime.datetime(2020,9,16))]
train3 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,31)) & (data['t_dat'] < datetime.datetime(2020,9,8))]
train4 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,23)) & (data['t_dat'] < datetime.datetime(2020,8,31))]

positive_items_per_user1 = train1.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user2 = train2.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user3 = train3.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user4 = train4.groupby(['customer_id'])['article_id'].apply(list)

train = pd.concat([train1, train2], axis=0)
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days)
popular_items_group = train.groupby(['article_id'])['pop_factor'].sum()

_, popular_items = zip(*sorted(zip(popular_items_group, popular_items_group.keys()))[::-1])

user_group = pd.concat([train1, train2, train3, train4], axis=0).groupby(['customer_id'])['article_id'].apply(list)

In [15]:
# SVD
train = pd.concat([train1, train2, train3, train4], axis=0)
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days**2)
popular_items_group = train.groupby(['article_id'])['pop_factor'].sum()

train['feedback'] = 1
train = train.groupby(['customer_id', 'article_id']).sum().reset_index()

train['feedback'] = train.apply(lambda row: row['feedback']/popular_items_group[row['article_id']], axis=1)

train['feedback'] = train['feedback'].apply(lambda x: 5.0 if x>5.0 else x)
train.drop(['price', 'sales_channel_id'], axis=1, inplace=True)
train['feedback'].describe()

count    1.020512e+06
mean     7.597183e-01
std      1.431037e+00
min      6.119724e-03
25%      4.992918e-02
50%      1.387718e-01
75%      5.250353e-01
max      5.000000e+00
Name: feedback, dtype: float64

In [16]:
train = train.sample(frac=1).reset_index(drop=True)
train.head()

,customer_id,article_id,pop_factor,feedback
0,3139bc28ef360db86ca0b7355b6edc41e683f6b40020e3...,0909916001,0.005540,0.125026
1,25886f645a47c7bdec126fef98455af0d395b716c38829...,0842605001,0.008264,0.294350
2,ee0c1bd5787ce0c1e1f2fb21e5e6e8a44fd63979032360...,0898692003,0.027778,0.028547
3,191071b64d4ae8584cc0cb6ce49d7d0cdd1da3cdfd37ff...,0872454002,0.001372,3.556682
4,e03d01d11c580a401db4692d43817da7f4dbfdd3e945b3...,0751471042,0.005917,0.048887


In [17]:
from reco.recommender import FunkSVD
from reco.metrics import rmse

f = FunkSVD(k=8, learning_rate=0.005, regularizer = .01, iterations = 200, method = 'stochastic', bias=True)
f.fit(X=train, formatizer={'user':'customer_id', 'item':'article_id', 'value':'feedback'},verbose=True)

Epoch 0: Error: 0.703984055986469
Epoch 1: Error: 0.5167095394743632
Epoch 2: Error: 0.44905168217005637
Epoch 3: Error: 0.40826668210792
Epoch 4: Error: 0.37953167919373515
Epoch 5: Error: 0.3576995449177035
Epoch 6: Error: 0.3403381645132104
Epoch 7: Error: 0.32599001996562155
Epoch 8: Error: 0.31374299845326487
Epoch 9: Error: 0.30303535785744207
Epoch 10: Error: 0.29351368702273756
Epoch 11: Error: 0.2849036813090378
Epoch 12: Error: 0.2770257893359142
Epoch 13: Error: 0.2697451393800591
Epoch 14: Error: 0.2629655877825135
Epoch 15: Error: 0.25660507939139793
Epoch 16: Error: 0.25060389683701584
Epoch 17: Error: 0.2449104950195829
Epoch 18: Error: 0.23948759349048832
Epoch 19: Error: 0.234303043457732
Epoch 20: Error: 0.22933402337176936
Epoch 21: Error: 0.22456471355573468
Epoch 22: Error: 0.21997858027858944
Epoch 23: Error: 0.21555953146443857
Epoch 24: Error: 0.2112959171229523
Epoch 25: Error: 0.2071814529184391
Epoch 26: Error: 0.20320830691709307
Epoch 27: Error: 0.199369410

In [18]:
submission = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [19]:
outputs = []
cnt = 0

popular_items = list(popular_items)
userindexes = {f.users[i]:i for i in range(len(f.users))}

for user in tqdm(submission['customer_id']):
    user_output = []
    if user in positive_items_per_user1.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user1[user]).most_common()}
        
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    if user in positive_items_per_user2.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user2[user]).most_common()}
        
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    if user in positive_items_per_user3.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user3[user]).most_common()}
        
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    if user in positive_items_per_user4.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user4[user]).most_common()}
        
        user_index = userindexes[user]
        new_order = {}
        for k in list(most_common_items_of_user.keys())[:20]:
            try:
                itemindex = f.items.index(k)
                pred_value = np.dot(f.userfeatures[user_index], f.itemfeatures[itemindex].T) + f.item_bias[0, itemindex]
            except:
                pred_value = most_common_items_of_user[k]
            new_order[k] = pred_value
        user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
        
    user_output += [pred_next[item] for item in user_output if item in pred_next and pred_next[item] not in user_output]      
    
    user_output += list(popular_items[:12 - len(user_output)])
    outputs.append(user_output)
    
str_outputs = []
for output in outputs:
    str_outputs.append(" ".join([str(x) for x in output]))

100%|██████████| 1371980/1371980 [11:27<00:00, 1996.98it/s]


In [20]:
submission['prediction'] = str_outputs
submission.to_csv("submissions.csv", index=False)

In [21]:
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0924243001 0924243002 0918522001 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0918522001 0751471001 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0918522001 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0918522001 0751471001 04...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0918522001 0751471001 04...
